In [60]:
import requests
import itertools
from arcgis.geometry import Geometry

import pandas as pd
from arcgis.features import SpatialDataFrame as SDF

from arcgis.gis import GIS, Item

import sys
sys.path.append('..')

import geometry_monkeypatch
import reach_tools

In [135]:
import importlib
importlib.reload(reach_tools)
importlib.reload(geometry_monkeypatch)

<module 'geometry_monkeypatch' from '../geometry_monkeypatch.py'>

In [136]:
waters = reach_tools.WATERS()
gis = GIS()

In [137]:
def update_trace_with_nhdplus(self):

    # get the snapped and corrected reach locations for the put-in
    self.putin.snap_to_nhdplus()

    # use the EPA navigate service to trace downstream
    trace_polyline = waters.get_downstream_trace_polyline(self.putin.nhdplus_reach_id, self.putin.nhdplus_measure)

    # project the takeout geometry to the same spatial reference as the trace polyline
    takeout_geom = self.takeout.geometry.match_spatial_reference(self.takeout.geometry)

    # snap the takeout geometry to the hydroline
    takeout_geom = takeout_geom.snap_to_line(trace_polyline)

    # update the takeout in the reach
    self.takeout.set_geometry(takeout_geom)

    # trim off the tail below the takeout
    self.geometry = trace_polyline.split_at_point(takeout_geom)[0]
    
    return True

reach_tools.Reach.update_trace_with_nhdplus = update_trace_with_nhdplus

In [138]:
ldub = reach_tools.Reach.get_from_aw(2156)
ldub.update_trace_with_nhdplus()

True

In [111]:
reach = ldub

geometry_field = 'hydroline'
df = pd.DataFrame(reach)
in_data = df[c for c in df.columns if c != geometry_field]
geometry = df[geometry_field]
sdf = SDF(data=in_data, geometry=geometry)
sdf.to_featureset()

SyntaxError: invalid syntax (<ipython-input-111-1e1c55e70fab>, line 5)

In [116]:
df = pd.DataFrame(reach)
df.columns

Index(['Willard to Columbia River'], dtype='object')

In [78]:
ldub.putin.geometry

{'x': -121.633094507918,
 'y': 45.7953236761081,
 'spatialReference': {'wkid': 4326}}

In [79]:
ldub.takeout.geometry

{'x': -121.645411521824,
 'y': 45.7183001959982,
 'spatialReference': {'wkid': 4326}}